In [12]:
import pandas as pd

In [13]:
df = pd.read_csv('data-stage2.csv')
df

,Unnamed: 0,category,amt,is_fraud,hour,trans_count_7d,trans_count_30d,time_diff
0,0,misc_net,4.97,0,1,0.0,0.0,0.000000
1,1,grocery_pos,107.23,0,1,0.0,0.0,0.000000
2,2,entertainment,220.11,0,1,0.0,0.0,0.000000
3,3,gas_transport,45.00,0,1,0.0,0.0,0.000000
4,4,misc_pos,41.96,0,1,0.0,0.0,0.000000
...,...,...,...,...,...,...,...,...
1852389,1852389,health_fitness,43.77,0,1,39.0,167.0,4.619444
1852390,1852390,kids_pets,111.84,0,1,62.0,272.0,2.706389
1852391,1852391,kids_pets,86.88,0,1,67.0,277.0,0.201111
1852392,1852392,travel,7.99,0,1,36.0,192.0,3.340278


In [14]:
df[['category']].value_counts()

category      
gas_transport     188029
grocery_pos       176191
home              175460
shopping_pos      166463
kids_pets         161727
shopping_net      139322
entertainment     134118
food_dining       130729
personal_care     130085
health_fitness    122553
misc_pos          114229
misc_net           90654
grocery_net        64878
travel             57956
dtype: int64

In [15]:
from sklearn.compose import ColumnTransformer
from sklearn.compose import make_column_selector
from sklearn.preprocessing import MinMaxScaler, OneHotEncoder
from sklearn import set_config

preprocessor = ColumnTransformer(
    transformers=[
        ("num", MinMaxScaler(), make_column_selector(dtype_include="float64")),
        ("cat", OneHotEncoder(), make_column_selector(dtype_include="object")),
    ],
    remainder='passthrough'
)

set_config(display="diagram")

preprocessor

ColumnTransformer(remainder='passthrough',
                  transformers=[('num', MinMaxScaler(),
                                 <sklearn.compose._column_transformer.make_column_selector object at 0x0000023CB2DB2A30>),
                                ('cat', OneHotEncoder(),
                                 <sklearn.compose._column_transformer.make_column_selector object at 0x0000023CB2DB2DC0>)])

In [19]:
X = preprocessor.fit_transform(df.drop(columns=['is_fraud']))
y = df[['is_fraud']]                        

In [25]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(
    X,
    y,
    test_size=0.2,
    stratify=y
)

print(X_train.shape, X_test.shape, y_train.shape, y_test.shape)

(1481915, 20) (370479, 20) (1481915, 1) (370479, 1)


In [21]:
y_train.value_counts()

is_fraud
0           1474194
1              7721
dtype: int64

In [22]:
y_test.value_counts()

is_fraud
0           368549
1             1930
dtype: int64

In [27]:
from imblearn.over_sampling import SMOTE

X_os, y_os = SMOTE().fit_resample(X_train, y_train)

print(X_os.shape, y_os.shape)

(2948388, 20) (2948388, 1)


In [28]:
from sklearn.ensemble import RandomForestClassifier

clf = RandomForestClassifier(n_jobs=-1)
clf.fit(X_us, y_us.to_numpy().reshape(-1))

RandomForestClassifier(n_jobs=-1)

In [32]:
from sklearn.metrics import classification_report

print("Oversampled Training Dataset")
print(classification_report(y_os, clf.predict(X_os)))

print("Original Training Dataset")
print(classification_report(y_train, clf.predict(X_train)))

print("Test Dataset")
print(classification_report(y_test, clf.predict(X_test)))

Oversampled Training Dataset
              precision    recall  f1-score   support

           0       1.00      1.00      1.00   1474194
           1       1.00      1.00      1.00   1474194

    accuracy                           1.00   2948388
   macro avg       1.00      1.00      1.00   2948388
weighted avg       1.00      1.00      1.00   2948388

Original Training Dataset
              precision    recall  f1-score   support

           0       1.00      1.00      1.00   1474194
           1       0.96      0.95      0.96      7721

    accuracy                           1.00   1481915
   macro avg       0.98      0.98      0.98   1481915
weighted avg       1.00      1.00      1.00   1481915

Test Dataset
              precision    recall  f1-score   support

           0       1.00      1.00      1.00    368549
           1       0.96      0.95      0.96      1930

    accuracy                           1.00    370479
   macro avg       0.98      0.98      0.98    370479
weight